In [10]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import RandomForestClassifier # import RandomForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [11]:
dfm = pd.read_csv('data/mushrooms.csv')
dfm.head()


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [12]:
dfm.isnull().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [13]:
x = dfm.iloc[:,1:]
y = dfm['class'] # ganti nama kolom dari 'diagnosis' menjadi 'class'
y = y.map({'p':1, 'e':0}) # Encode label

x.shape

(8124, 22)

In [14]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=1)

In [16]:
x = pd.get_dummies(x) # konversi kolom kategorial ke numeric

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=1)

dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)

# predict
y_pred_dt = dt.predict(x_test)

#calculate accuracy
acc_dt = accuracy_score(y_test,y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt))
print(f"test set accuracy: {acc_dt}")

Test set accuracy: 1.00
test set accuracy: 1.0


In [17]:
from sklearn.ensemble import AdaBoostClassifier
# AdaBoost Training
ada = AdaBoostClassifier(n_estimators=2)

# Sesuaikan dt ke set training
ada.fit(x_train, y_train)

# Memprediksi label set test
y_pred_ada = ada.predict(x_test)

#  menghitung set accuracy
acc_ada = accuracy_score(y_test, y_pred_ada)
print("Test set accuracy: {:.2f}".format(acc_ada))
print(f"Test set accuracy: {acc_ada}")

Test set accuracy: 0.88
Test set accuracy: 0.88


c:\Users\WAYCOM\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


# Tugas 3


In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Memuat dataset dari file CSV
dbt = pd.read_csv('data/diabetes.csv')

# Memisahkan fitur dan target
X = dbt.iloc[:, :-1]  # Semua kolom kecuali kolom terakhir sebagai fitur
y = dbt.iloc[:, -1]   # Kolom terakhir sebagai target

# Membagi dataset menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardisasi fitur
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Membangun model
logreg = LogisticRegression(max_iter=1000)
svm_model = SVC(kernel='poly')
tree_model = DecisionTreeClassifier()

# Tuning Hyperparameter
# a. Tuning  Logistic Regression
logreg_params = {'C': [0.01, 0.1, 1, 10, 100]}
logreg_grid = GridSearchCV(logreg, logreg_params, cv=5)
logreg_grid.fit(X_train_scaled, y_train)
logreg_best = logreg_grid.best_estimator_

# b. Tuning  SVM
svm_params = {'C': [0.01, 0.1, 1, 10, 100], 'degree': [2, 3, 4]}
svm_grid = GridSearchCV(svm_model, svm_params, cv=5)
svm_grid.fit(X_train_scaled, y_train)
svm_best = svm_grid.best_estimator_

# c. Tuning  Decision Tree
tree_params = {'max_depth': [None, 5, 10, 15, 20], 'min_samples_split': [2, 5, 10]}
tree_grid = GridSearchCV(tree_model, tree_params, cv=5)
tree_grid.fit(X_train, y_train)
tree_best = tree_grid.best_estimator_

# Membangun Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('logreg', logreg_best),
    ('svm', svm_best),
    ('tree', tree_best)],
    voting='hard')

# Melatih Voting Classifier
voting_clf.fit(X_train_scaled, y_train)

# Evaluasi Model
y_pred = voting_clf.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Voting Classifier Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Voting Classifier Accuracy: 0.7727272727272727
              precision    recall  f1-score   support

           0       0.81      0.85      0.83        99
           1       0.70      0.64      0.67        55

    accuracy                           0.77       154
   macro avg       0.75      0.74      0.75       154
weighted avg       0.77      0.77      0.77       154

